In [1]:
import pandas as pd
import numpy as np
import math
import os

# load txt data
data = pd.read_csv('test_text.txt', sep="\t", header=None, encoding='utf-8')
df = data.copy()

df.rename(columns={0:'text'}, inplace=True)

df['fog_word3'] = np.nan   
df['fog_word5'] = np.nan
df['fog_word7'] = np.nan
df['fog_word10'] = np.nan
df['flesch'] = np.nan
df['length'] = np.nan

In [2]:
df

,text,fog_word3,fog_word5,fog_word7,fog_word10,flesch,length
0,개요: 본 연구에서는 영문 기반의 사업보고서의 가독성을 측정한 기존 연구의 결과가 ...,NaN,NaN,NaN,NaN,NaN,NaN
1,회계는 경영의 언어이다. 회계는 재무정보의 생산자(내부자)와 소비자(외부자)가 소통...,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.iloc[0, 0]

'개요: 본 연구에서는 영문 기반의 사업보고서의 가독성을 측정한 기존 연구의 결과가 우리말 사업보고서에도 확장가능한지 가늠해보기로 한다. Li(2008)의 연구 이후, 회계와 재무분야의 많은 연구에서 사업보고서의 가독성을 비롯한 기업공시의 언어적 특성(linguistic features)에 대한 연구가 폭넓게 이루어졌으나, 상이한 전자공시체계로 인한 분석의 용이성 차이 때문에 국내에서는 이와 관련된 연구가 미비한 상황이다. 본 연구는 Li(2008)의 방법론을 차용하여 국내 상장기업들을 대상으로 우리말 사업보고서의 가독성을 측정하고,이익특성 변수들 간의 관계를 확인하고자 하였다. 주요 분석결과에 따르면, 상대적으로 낮은 이익을 보고하는 기업의 경우 사업보고서의 가독성이 낮은 것으로 나타났다. 또한, 사업보고서의 가독성이 낮은 기업의 경우, 이익의 지속성 역시 떨어진다는 점을 확인하였다. 마지막으로 이익조정 유인이 큰 기업들의 사업보고서의 가독성이 떨어진다는 점을 확인하였다. 이는 영문 기반의 사업보고서 가독성을 분석한 Li(2008)의 연구 및 후속 연구들의 결과가 우리말 기반의 사업보고서에서도 동일하게  적용될 수 있는 가능성을 시사한다. 본 연구는 우리말 사업보고서의 가독성과 관련된 텍스트 분석의 확장 가능성을 체계적으로 제시하였다는 점에서 의미가 있다.'

In [4]:
for i in range(df.shape[0]):
    raw_data = df['text'][i]

    letter_num = 0    # 글자수
    word3_num = 0     # 3음절 이상 단어 수
    word5_num = 0     # 5음절 이상 단어 수
    word7_num = 0     # 7음절 이상 단어 수
    word10_num = 0    # 10음절 이상 단어 수
    word_num = 0      # 단어 수
    sentence_num = 0  # 문장 수

    for word in range(len(raw_data.split(" "))):   # 단어 수는 빈칸(" ")으로 구분
        # 글자 수 또는 음절 수는 단어를 구성하는 요소로 판단함
        letter_num += len(raw_data.split(" ")[word])  # 단어 하나의 글자 수를 더함
        word_num += 1
        if len(raw_data.split(" ")[word]) > 2:
            word3_num += 1
        if len(raw_data.split(" ")[word]) > 4:
            word5_num += 1
        if len(raw_data.split(" ")[word]) > 6:
            word7_num += 1
        if len(raw_data.split(" ")[word]) > 9:
            word10_num += 1

    for sentence in range(len(raw_data.split("."))):  # 문장 수는 마침표로 구분
        split_length = len(raw_data.split(".")[sentence])
        if split_length == 0:
            sentence_num = sentence_num
        elif raw_data.split(".")[sentence][split_length - 1].isdigit():  # 마침표 앞에 숫자면 제외, ex) 10.31%, 17.08.11 등
            sentence_num = sentence_num
        else:  # 위의 경우를 제외하고, 문장 수 집계
            sentence_num += 1
            
    AWL = letter_num / word_num     # Flesch 구성요소 : 평균 단어 길이
    ASL = word_num / sentence_num   # Flesch 구성요소 : 평균 문장 길이
    fog_word3 = (ASL + (word3_num / word_num) * 100) * 0.4  # Fog index 측정
    fog_word5 = (ASL + (word5_num / word_num) * 100) * 0.4
    fog_word7 = (ASL + (word7_num / word_num) * 100) * 0.4
    fog_word10 = (ASL + (word10_num / word_num) * 100) * 0.4
    flesch = 206.835 - 84.6 * AWL - 1.015 * ASL             # Flesch 측정
    length = math.log(word_num)     # 문서의 길이 측정

    df.loc[i, 'fog_word3'] = fog_word3
    df.loc[i, 'fog_word5'] = fog_word5
    df.loc[i, 'fog_word7'] = fog_word7
    df.loc[i, 'fog_word10'] = fog_word10
    df.loc[i, 'flesch'] = flesch
    df.loc[i, 'length'] = length

In [5]:
df

,text,fog_word3,fog_word5,fog_word7,fog_word10,flesch,length
0,개요: 본 연구에서는 영문 기반의 사업보고서의 가독성을 측정한 기존 연구의 결과가 ...,37.798905,17.944891,9.769708,7.433942,-131.656364,4.919981
1,회계는 경영의 언어이다. 회계는 재무정보의 생산자(내부자)와 소비자(외부자)가 소통...,42.109524,21.752381,12.645238,9.430952,-189.790238,5.411646
